In [ ]:
import json
import os
import warnings

import pandas as pd
import pytz

from youtube.config import ignore_videos_longer_than_seconds, \
    assume_all_videos_were_watched_to_factor, ignore_videos_from_channel_regex, youtube_watch_history_file, \
    video_details_download_folder, output_dir_unique, year_to_analyze

warnings.filterwarnings('ignore')

def parse_to_seconds(duration_iso):
    try:
        if 'H' in duration_iso:
            hours = int(duration_iso.split('H')[0].split('T')[1]) if duration_iso.split('H')[0].split('T')[1] else 0
            minutes = int(duration_iso.split('H')[1].split('M')[0]) if duration_iso.split('H')[1].split('M')[0] else 0
            seconds = int(duration_iso.split('M')[1].split('S')[0]) if duration_iso.split('M')[1].split('S')[0] else 0
            return hours * 3600 + minutes * 60 + seconds
        elif 'M' in duration_iso:
            minutes = int(duration_iso.split('M')[0].split('T')[1]) if duration_iso.split('M')[0].split('T')[1] else 0
            seconds = int(duration_iso.split('M')[1].split('S')[0]) if duration_iso.split('M')[1].split('S')[0] else 0
            return minutes * 60 + seconds
        else:
            return int(duration_iso.split('S')[0].split('T')[1]) if duration_iso.split('S')[0].split('T')[1] else 0
    except (IndexError, ValueError):
        return -1

df = pd.read_json(youtube_watch_history_file)

# extract video_id from titleUrl
df['video_id'] = df['titleUrl'].str.extract(r'v=(.*)')
# keep rows where 'details' column is NaN which means not Ads
df = df[df['details'].isna()].reset_index()

# load video details from raw file
def load_all_video_details(directory):
    tmp = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            video_id = os.path.splitext(filename)[0]
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                video_details = json.load(file)
                tmp[video_id] = video_details
    return tmp

video_details_map = load_all_video_details(video_details_download_folder)

def get_duration_seconds(video_id):
    if video_id in video_details_map:
        items = video_details_map[video_id].get('items', [])
        if items:
            return parse_to_seconds(items[0]['contentDetails']['duration'])
    return None

def get_channel_url(video_id):
    if video_id in video_details_map:
        items = video_details_map[video_id].get('items', [])
        if items:
            channel_id = items[0]['snippet']['channelId']
            return f'https://www.youtube.com/channel/{channel_id}'
    return None

# Ensure the 'time' column is of datetime type
df['time'] = pd.to_datetime(df['time'], errors='coerce')
# Convert the 'time' column to Berlin timezone
berlin_tz = pytz.timezone('Europe/Berlin')
df['time'] = df['time'].dt.tz_convert(berlin_tz)
# Add weekday and minutes played columns
df['weekday'] = df['time'].dt.day_name()
df['year'] = df['time'].dt.year

df['duration_seconds'] = round(df['video_id'].map(get_duration_seconds))
df['duration_seconds'] = assume_all_videos_were_watched_to_factor * df['duration_seconds']

df['duration_minutes'] = round(df['duration_seconds'] / 60)
df['duration_hours'] = round(df['duration_seconds'] / 3600)

df['channel_url'] = df['video_id'].map(get_channel_url)
df['video_url'] = df['video_id'].map(lambda x: f'https://www.youtube.com/watch?v={x}')

# Add hour of the day
df['hour'] = df['time'].dt.hour

# Remove videos that are longer than 2 hours (7200 seconds)
df = df[df['duration_seconds'] <= ignore_videos_longer_than_seconds]

# remove videos if year_to_analyze is set
if year_to_analyze != 'all':
    df = df[df['year'] == int(year_to_analyze)]

# add channel title
df['channel_title'] = df['video_id'].map(lambda x: video_details_map[x]['items'][0]['snippet']['channelTitle'])

# remove all videos that match ignore_videos_from_channel_regex
for regex in ignore_videos_from_channel_regex:
    df = df[~df['channel_title'].str.contains(regex, case=False)]

# remove all videos that have duration_seconds NaN which means the video is not available anymore
df = df[~df['duration_seconds'].isna()]
# remove header column
df = df.drop(columns=['header', 'titleUrl', 'subtitles', 'products', 'activityControls', 'details', 'index', 'description'])


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Calculate the total watched hours per year
df['watched_hours'] = df['duration_seconds'] / 3600
total_watched_hours_per_year = df.groupby('year')['watched_hours'].sum().reset_index()

# Plot the results
plt.figure(figsize=(10, 5))
plt.bar(total_watched_hours_per_year['year'].astype(str), total_watched_hours_per_year['watched_hours'], color='skyblue')
plt.title('Total Watched Hours Per Year')
plt.xlabel('Year')
plt.ylabel('Total Watched Hours')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir_unique, 'total_watched_hours_per_year.png'))
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure the 'time' column is of datetime type
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Extract the year and month from the 'time' column
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month

# Calculate the total watched hours per month for each year
df['watched_hours'] = df['duration_seconds'] / 3600
total_watched_hours_per_month = df.groupby(['year', 'month'])['watched_hours'].sum().reset_index()

# Pivot the data to have years as columns and months as rows
pivot_df = total_watched_hours_per_month.pivot(index='month', columns='year', values='watched_hours')

# Plot the results
pivot_df.plot(kind='line', marker='o', figsize=(12, 6))
plt.title('Total Watched Hours Per Month for Each Year')
plt.xlabel('Month')
plt.ylabel('Total Watched Hours')
plt.xticks(range(1, 13))
plt.grid(True)
plt.legend(title='Year')
plt.tight_layout()
plt.savefig(os.path.join(output_dir_unique, 'total_watched_hours_per_month.png'))
plt.show()

In [ ]:

import matplotlib.pyplot as plt

# Group by channel_title and sum the duration_seconds
channel_grouped = df.groupby('channel_title')['duration_seconds'].sum().reset_index()

# Convert duration_seconds to hours
channel_grouped['duration_hours'] = channel_grouped['duration_seconds'] / 3600

# Sort the grouped data by duration_hours in descending order
top_channels = channel_grouped.sort_values(by='duration_hours', ascending=False).head(30)

# Plot the data
plt.figure(figsize=(12, 8))
plt.barh(top_channels['channel_title'], top_channels['duration_hours'], color='skyblue')
plt.xlabel('Hours Streamed')
plt.ylabel('Channel Title')
plt.title('Top Channels by Hours Stream Time')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest value on top
plt.savefig(os.path.join(output_dir_unique, 'top_channels_by_hours_streamed.png'), bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Ensure the 'time' column is of datetime type
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Drop rows with NaT values in the 'time' column
df = df.dropna(subset=['time'])

# Group by week and count the number of videos streamed
df['week'] = df['time'].dt.to_period('W').apply(lambda r: r.start_time)
videos_per_week = df.groupby('week').size().reset_index(name='video_count')

# Plot the data
plt.figure(figsize=(12, 6))
plt.bar(videos_per_week['week'], videos_per_week['video_count'], color='skyblue')
plt.xlabel('Week')
plt.ylabel('Number of Videos Streamed')
plt.title('Number of Videos Streamed per Week')
plt.xticks(rotation=45)
plt.ylim(0, 150)  # Limit y-axis to 150 videos
plt.grid(True)

# Add trendline
z = np.polyfit(range(len(videos_per_week)), videos_per_week['video_count'], 1)
p = np.poly1d(z)
plt.plot(videos_per_week['week'], p(range(len(videos_per_week))), color='red', linestyle='--')
plt.savefig(os.path.join(output_dir_unique, 'videos_streamed_per_week.png'), bbox_inches='tight')
plt.show()

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# Calculate hours played
df['duration_hours'] = df['duration_seconds'] / 3600

# Group by day and calculate the total hours played for each day
df['date'] = df['time'].dt.date
daily_hours = df.groupby('date')['duration_hours'].sum().reset_index()

# Calculate the 95th percentile value
percentile_95 = daily_hours['duration_hours'].quantile(0.95)

# Filter the DataFrame to only keep rows within the 95th percentile
filtered_daily_hours = daily_hours[daily_hours['duration_hours'] <= percentile_95]

# Calculate the average hours played per day for the filtered data
average_daily_hours = filtered_daily_hours['duration_hours'].mean()

# Define custom colormap
colors = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
cmap = mcolors.ListedColormap(colors)
bounds = np.arange(0, len(colors) + 1, 1)
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# Plot the data
plt.figure(figsize=(12, 6))
sc = plt.scatter(filtered_daily_hours['date'], filtered_daily_hours['duration_hours'], c=filtered_daily_hours['duration_hours'], cmap=cmap, norm=norm, edgecolors='black')
plt.colorbar(sc, ticks=bounds)
plt.axhline(y=average_daily_hours, color='red', linestyle='--', label=f'Average: {average_daily_hours:.2f} hours/day')

# Add trendline
z = np.polyfit(range(len(filtered_daily_hours)), filtered_daily_hours['duration_hours'], 1)
p = np.poly1d(z)
plt.plot(filtered_daily_hours['date'], p(range(len(filtered_daily_hours))), color='blue', linestyle='--', label='Trendline')

plt.xlabel('Date')
plt.ylabel('Hours Played')
plt.title('Average Hours Played per Day (Filtered to 95th Percentile)')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir_unique, 'average_hours_played_per_day.png'), bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd

# Ensure the 'time' column is of datetime type
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Extract the year from the 'time' column
df['year'] = df['time'].dt.year

# Group by year and channel_title, and count the number of videos
year_channel_video_count = df.groupby(['year', 'channel_title']).size().reset_index(name='video_count')

# Sort the grouped data by year and video_count in descending order
sorted_year_channel_video_count = year_channel_video_count.sort_values(by=['year', 'video_count'], ascending=[False, False])

# Keep only the top ten channels per year
top_ten_per_year = sorted_year_channel_video_count.groupby('year').head(10).reset_index(drop=True)

# Add the channel_url to the result
top_ten_per_year['channel_url'] = top_ten_per_year['channel_title'].map(lambda x: df[df['channel_title'] == x]['channel_url'].values[0])

top_ten_per_year.to_csv(os.path.join(output_dir_unique, 'top_ten_channels_per_year.csv'), index=False)

# Display the result
top_ten_per_year

In [ ]:
import pandas as pd

# Ensure the 'time' column is of datetime type
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Extract the year from the 'time' column
df['year'] = df['time'].dt.year

# Sort the DataFrame by 'year' (descending) and 'channel_title'
sorted_videos = df.sort_values(by=['year', 'channel_title'], ascending=[False, True])

# Add the video_url column
sorted_videos['video_url'] = sorted_videos['video_id'].map(lambda x: f'https://www.youtube.com/watch?v={x}')

# Display the result
sorted_videos_list = sorted_videos[['year', 'channel_title', 'title', 'video_url']].reset_index(drop=True)
sorted_videos_list.to_csv(os.path.join(output_dir_unique, 'all_videos_by_year_channel.csv'), index=False)
sorted_videos_list

In [ ]:
import pandas as pd

# Ensure the 'time' column is of datetime type
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Group by video_id and count the occurrences
video_play_counts = df.groupby('video_id').size().reset_index(name='play_count')

# Filter to include only videos played more than once
multiple_plays = video_play_counts[video_play_counts['play_count'] > 1]

# Merge with the original DataFrame to get additional video details
multiple_plays_details = multiple_plays.merge(df, on='video_id', how='left')

# Sort by play_count in descending order
multiple_plays_details = multiple_plays_details.sort_values(by='play_count', ascending=False)

# Display the result
multiple_plays_list = multiple_plays_details[['year', 'title', 'channel_title', 'play_count', 'video_url']].reset_index(drop=True)
multiple_plays_list.to_csv(os.path.join(output_dir_unique, 'videos_played_multiple_times.csv'), index=False)

multiple_plays_list